# Import

In [2]:
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

from src.config import *
from src.models.model_util import *
from src.data.data_util import *
from src.visualization.auggrad import *
from src.visualization.cams import *
from src.visualization.visual_utils import *
from src.visualization.vistool_pipeline import *

from IPython.display import clear_output

In [3]:
# LOAD CONFIG
cfg = load_config()

# MODEL CONFIG
cfg.model_name = 'resnet_no_maxpool'
cfg.start_channels = 64
model, cfg.device = load_model(cfg.model_name, verbose=False, cfg=cfg)
model.load_state_dict(torch.load('./result/models/2021_04_07_1408_resnet_no_maxpool/resnet_no_maxpool_ep70-100_sd42_mae6.241.pth/'))

# Dataloader
# cfg.return_path = True
train_dataset = DatasetPlus(cfg, augment=False, test=True)
sample_dl = DataLoader(train_dataset, batch_size=1)

# FOR SINGLE BRAIN
# x, y = train_dataset[15]
# x = x[None, ...]

In [ ]:
gcam = VisTool(cfg, model, cam_type='gcam')
path = './result/models/2021_04_07_1408_resnet_no_maxpool/*.pth'
gcam.run_pretrains_dataloader(path, sample_dl, layer_index=8, visualize=False)

In [ ]:
sgrad = VisTool(cfg, model, cam_type='sgrad')
path = './result/models/2021_04_07_1408_resnet_no_maxpool/*.pth'
sgrad.run_pretrains_dataloader(path, sample_dl, visualize=False)

In [ ]:
agrad = VisTool(cfg, model, cam_type='agrad')
path = './result/models/2021_04_07_1408_resnet_no_maxpool/*.pth'
agrad.run_pretrains_dataloader(path, sample_dl, visualize=False)

In [ ]:
# make averaged_brain + vismap
averaged_brain = F.interpolate(torch.tensor(np.load('../../brainmask_tlrc/averaged_brain_no_oas3.npy'))[None, None, ...],
                              size=(96, 96, 96))
for i, v in enumerate(gcam.vismap_ts):
    plot_vismap(brain=averaged_brain, vismap=torch.tensor(v[0]), save=True)